In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

# Test
print(os.getenv("CLIMATETRACE_DB"))

climatetrace


In [ ]:
# grabbing data from asset table

asset_query = '''
    select ca.name country_name
        , ae.iso3_country
        , original_inventory_sector
        , to_char(start_time, 'YYYY-MM') as year_month
        , ae.gas
        , sum(activity) activity
        , avg(emissions_factor) mean_emissions_factor
        , sum(emissions_quantity) emissions_quantity

    from asset_emissions ae
    left join country_analysis ca
        on cast(ca.iso3_country as varchar) = cast(ae.iso3_country as varchar)

    where extract(year from start_time) >= 2023
        and ae.gas = 'co2e_100yr'
        and original_inventory_sector in ('international-shipping','domestic-shipping')
        and most_granular = true

    group by ca.name
        , ae.iso3_country
        , original_inventory_sector
        , to_char(start_time, 'YYYY-MM')
        , end_time
        , ae.gas
'''

asset_df = run_sql(asset_query)

,country_name,iso3_country,original_inventory_sector,month,year,gas,activity,mean_emissions_factor,emissions_quantity
0,Afghanistan,AFG,international-shipping,3.0,2023.0,co2e_100yr,1361.317505,3.423796,4660.872955
1,Åland Islands,ALA,domestic-shipping,1.0,2023.0,co2e_100yr,7204.848858,0.705819,4304.225057
2,Åland Islands,ALA,domestic-shipping,1.0,2024.0,co2e_100yr,5656.494703,0.484072,2917.757603
3,Åland Islands,ALA,domestic-shipping,1.0,2025.0,co2e_100yr,10034.407117,0.916910,7496.852820
4,Åland Islands,ALA,domestic-shipping,2.0,2023.0,co2e_100yr,10260.580337,0.551878,2555.613160
...,...,...,...,...,...,...,...,...,...
11176,Yemen,YEM,international-shipping,10.0,2024.0,co2e_100yr,62981.106150,0.152232,10360.201245
11177,Yemen,YEM,international-shipping,11.0,2023.0,co2e_100yr,80698.537400,0.130066,12000.276583
11178,Yemen,YEM,international-shipping,11.0,2024.0,co2e_100yr,67711.491604,0.138828,12158.820812
11179,Yemen,YEM,international-shipping,12.0,2023.0,co2e_100yr,71366.794283,0.112487,11050.371500


In [18]:
asset_df.to_csv('data/country_monthly/month_country_subsector.csv', index=False)

In [38]:
import pandas as pd

df = pd.read_csv('data/country_monthly/month_country_subsector.csv')

df['year_month'] = df['year'].astype(int) + '-' + df['month'].astype(int).apply(lambda x: f'{x:02d}')


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [36]:
global_df = df.groupby('year_month').agg({
    'activity': 'sum',
    'mean_emissions_factor': 'mean',
    'emissions_quantity': 'sum'
}).reset_index()

In [30]:
df.columns

Index(['country_name', 'iso3_country', 'original_inventory_sector', 'month',
       'year', 'gas', 'activity', 'mean_emissions_factor',
       'emissions_quantity', 'month_year'],
      dtype='object')

In [37]:
global_df

,year_month,activity,mean_emissions_factor,emissions_quantity
0,2023.0-01,2.753425e+08,0.693745,7.885502e+07
1,2023.0-02,2.597545e+08,0.648406,7.313315e+07
2,2023.0-03,2.971428e+08,0.761522,8.283862e+07
3,2023.0-04,2.949980e+08,0.782434,8.381884e+07
4,2023.0-05,2.877300e+08,0.800155,8.584268e+07
5,2023.0-06,2.872027e+08,0.689101,8.565950e+07
6,2023.0-07,2.938026e+08,0.737569,8.744659e+07
7,2023.0-08,3.025805e+08,2.720913,8.762864e+07
8,2023.0-09,2.993846e+08,0.669918,8.390137e+07
9,2023.0-10,2.990533e+08,0.623143,8.363902e+07
